# MenuNER

In [ ]:
%cd /content
%rm -rf MenuNER
!git clone https://github.com/eleonorapoeta/MenuNER.git
%cd MenuNER
!git checkout origin/develop

## Installing requirements
### Changing to the MenuNER directory

In [ ]:
! pip install -r requirements.txt

In [ ]:
% cd /content/MenuNER

## Imports

In [ ]:
import json
from tqdm import tqdm
import logging
import torch.cuda
from torch.utils.data import DataLoader
import random
from ner.preprocess import dataner_preprocess
from ner.dataset import MenuDataset
from model import BiLSTM_CRF
from ner.util_preprocess import pos2ix, convert_examples_to_feature
from transformers import AdamW, get_linear_schedule_with_warmup
from datasets.metric import temp_seed
from torch.cuda.amp import GradScaler
from ner.util import to_device, evaluation
from seqeval.metrics import precision_score, f1_score, recall_score
from easydict import EasyDict as edict

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Train

### Here is not used the Argument parser but all the arguments are inserted in the following dictionary.

In [ ]:
opt = edict({
        'data_dir' : '/content/MenuNER/data/menu',
        'bert_checkpoint' : '/content/drive/MyDrive/DNLP_Polito/Dataset/Embeddings/bert_plus_eps',
        'seed' : 1000,
        'train' : 'train.txt',
        'max_length' : 512,
        'test' : 'test.txt',
        'val' : 'valid.txt',
        'pos' : True,
        'char' : True,
        'attention' : True,
        'pos_embedding_dim' : 64,
        'char_embedding_dim' : 25,
        'lr' : 1e-3,
        'epochs' : 30,
        'eval_and_save_steps' : 300
        })

### Creating the Train and Valid Dataset

In [ ]:
# set seed
random.seed(opt.seed)

# Processing of data and creation of features - TRAIN/TEST/VAL

train_examples = dataner_preprocess(opt.data_dir, opt.train)
val_examples = dataner_preprocess(opt.data_dir, opt.val)

# Create Feature
input_feats_train = convert_examples_to_feature(train_examples)
input_feats_val = convert_examples_to_feature(val_examples)

# Create train_loader and val_loader

dataset_train = MenuDataset(input_feats_train)
dataset_val = MenuDataset(input_feats_val)


train_loader = DataLoader(dataset=dataset_train,
                          batch_size=8,
                          shuffle=True)

valid_loader = DataLoader(dataset=dataset_val,
                          batch_size=8,
                          shuffle=True)

### Main

In [ ]:
def main():

    if torch.cuda.is_available():
      logger.info("%s", torch.cuda.get_device_name(0))
      device = torch.device('cuda')
    else:
      device = torch.device('cpu')

    tag_to_idx = {"PAD" : 0, "MENU" : 1, "O": 2}
    with temp_seed(opt.seed):
      model = BiLSTM_CRF(tagset_size=len(tag_to_idx),
                           bert_checkpoint=opt.bert_checkpoint,
                           max_length=opt.max_length,
                           embedding_dim=768,
                           hidden_dim=512,
                           pos2ix=pos2ix(train_examples),
                           pos_embedding_dim=opt.pos_embedding_dim,
                           pos=opt.pos,
                           char=opt.char,
                           char_embedding_dim=opt.char_embedding_dim,
                           attention=opt.attention).to(device)

      # create optimizer, scheduler, scaler, early_stopping
      optimizer = AdamW(params=model.parameters(),
                        lr=1e-3,
                        eps=1e-8,
                        weight_decay=0.01)

      num_training_steps_per_epoch = len(train_loader)
      num_training_steps = num_training_steps_per_epoch * opt.epochs

      scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                                  num_warmup_steps=0,
                                                  num_training_steps=num_training_steps)

      scaler = GradScaler()

      best_f1_score = - float('inf')
      best_f1_score_epoch = 0
      path_checkpoint = './train_model_checkpoint.pt'  # modified
      for epoch in range(opt.epochs):
          train_loss = 0.
          local_best_eval_loss = float('inf')
          local_best_eval_f1 = 0

          optimizer.zero_grad()

          for step, (x, y) in enumerate(tqdm(train_loader, total=len(train_loader), desc=f"Epoch {epoch}")):
              model.train()
              x = to_device(x, device)
              y = to_device(y, device)

              mask = torch.sign(torch.abs(x[1])).to(torch.uint8)
              logits, predictions = model(x)
              log_likelihood = model.crf_module(logits, y, mask=mask, reduction='mean')

              loss = log_likelihood * (-1)

              # backpropagation
              if device == 'cpu':
                  loss.backward()
              else:
                  scaler.scale(loss).backward()

              if device == 'cpu':
                  optimizer.step()
              else:

                  scaler.unscale_(optimizer)
                  torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                  scaler.step(optimizer)
                  scaler.update()

              optimizer.zero_grad()
              scheduler.step()

              print(f"Epoch {epoch}, local_step: {step}, loss: {loss:.3f}, curr_lr: {scheduler.get_last_lr()[0]:.7f}")
              global_step = (len(train_loader) * epoch) + step

              if opt.eval_and_save_steps > 0 and global_step != 0 and global_step % opt.eval_and_save_steps == 0:
                  report = evaluation(model, valid_loader, device, epoch)

                  if local_best_eval_loss > report['loss']:
                      local_best_eval_loss = report['loss']

                  if local_best_eval_f1 < report['f1']:
                      local_best_eval_f1 = report['f1']

                  if report['f1'] > best_f1_score:
                      best_f1_score = report['f1']
                      with open(path_checkpoint, 'wb') as f:
                          checkpoint = model.state_dict()
                          torch.save(checkpoint, f)

              train_loss += loss.item()

          # Evaluate in each epoch
          report = evaluation(model, valid_loader, device, epoch)

          if local_best_eval_loss > report['loss']:
              local_best_eval_loss = report['loss']

          if local_best_eval_f1 < report['f1']:
              local_best_eval_f1 = report['f1']

          if report['f1'] > best_f1_score:
              best_f1_score = report['f1']
              best_f1_score_epoch = epoch
              with open(path_checkpoint, 'wb') as f:
                  checkpoint = model.state_dict()
                  torch.save(checkpoint, f)

          logs = {
              'epoch': epoch,
              'local_step': step + 1,
              'epoch_step': len(train_loader),
              'local_best_eval_loss': local_best_eval_loss,
              'local_best_eval_f1': local_best_eval_f1,
              'best_eval_f1': best_f1_score,
          }

          logger.info(json.dumps(logs, indent=4))
          if (epoch - best_f1_score_epoch) >= 7:  # Our Early stopping
              print(f"Early stopped Training at epoch:{epoch}")
              break

In [ ]:
main()

## Test

In [ ]:
opt = edict({
        'data_dir' : '/content/MenuNER/data/menu',
        'test' : 'test.txt',
        'pos2ix' : '/content/MenuNER/data/pos2ix_dict.json',
        'model_path' : '/content/MenuNER/data/model_checkpoint.pt',
        'bert_checkpoint' : '/content/MenuNER/data/bert_checkpoint',
        'max_length' : 512,
        'pos' : True,
        'char' : True,
        'attention' : True,
        'pos_embedding_dim' : 64,
        'char_embedding_dim' : 25
      })

### Test Dataset creation

In [ ]:
test_examples = dataner_preprocess(opt.data_dir, opt.test)
input_feats_test = convert_examples_to_feature(test_examples)
dataset_test = MenuDataset(input_feats_test)
test_loader = DataLoader(dataset=dataset_test,
                          batch_size=8,
                          shuffle=True)

### Test

In [ ]:
tag_to_idx = {"PAD": 0, "MENU": 1, "O": 2}

def test():

    test_examples = dataner_preprocess(opt.data_dir, opt.test)
    input_feats_test = convert_examples_to_feature(test_examples)
    dataset_val = MenuDataset(input_feats_test)
    valid_loader = DataLoader(dataset=dataset_val,
                              batch_size=8,
                              shuffle=True)

    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    with open(opt.pos2ix, 'r') as f:
        pos2ix = json.load(f)

    model = BiLSTM_CRF(tagset_size=len(tag_to_idx),
                       bert_checkpoint=opt.bert_checkpoint,
                       max_length=opt.max_length,
                       embedding_dim=768,
                       hidden_dim=512,
                       pos2ix=pos2ix,
                       pos=opt.pos,
                       char=opt.char,
                       pos_embedding_dim=opt.pos_embedding_dim,
                       char_embedding_dim=opt.char_embedding_dim,
                       attention=opt.attention).to(device)

    checkpoint = torch.load(opt.model_path)
    model.load_state_dict(checkpoint)

    report = evaluation(model, valid_loader, device)

    logs = {
        'precision': report['precision'],
        'f1': report['f1'],
        'recall': report['recall']
    }

    logger.info(json.dumps(logs, indent=4))

    print(report)